In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp

from deeplp.models.data_prep import create_weighted_graph, load_data
from deeplp.models.data_prep import prepare_data, random_unlabel, calc_masks
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.models.utils import accuracy, indices_to_vec
from deeplp.models.data_prep import select_features


/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def approx_chunk(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [16]:
accuracies = np.zeros((3,10,10))
for i,data in enumerate(['flickr']):

    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'linqs',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

        for j in range(1,11):
            print(data,seed,j)
            final_accs = []

            labeled_indices_copy = copy.copy(labeled_indices)
            random.seed(seed)
            shuffle(labeled_indices_copy)
            cv_held_out_indices_list = approx_chunk(labeled_indices_copy, 5)


            for k, cv_held_out_indices in enumerate(cv_held_out_indices_list):
                
                cv_labeled_indices = [index for index in labeled_indices if index not in cv_held_out_indices]
                cv_unlabeled_indices = np.delete(np.arange(true_labels.shape[0]),cv_labeled_indices)
                cv_labels, cv_is_labeled = calc_masks(true_labels, cv_labeled_indices, cv_unlabeled_indices,logistic=0)
                lp = LP()
                index = np.hstack([cv_held_out_indices,cv_unlabeled_indices])
                unlabeled_pred = lp.iter_sp(cv_labels,
                                     graph,
                                     cv_is_labeled,
                                     10 * j,
                                     index)
                
                y_pred = np.argmax(unlabeled_pred,axis=1)
                y_true = np.argmax(true_labels[index],axis=1)
                acc = np.mean(y_pred == y_true)
                final_accs.append(acc)
            acc = np.mean(final_accs)
            accuracies[i,seed,j-1] = acc
            print("baseline",acc)


-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
cora 0 1
baseline 0.651497975708502
cora 0 2
baseline 0.6536032388663967
cora 0 3
baseline 0.6551417004048583
cora 0 4
baseline 0.6550607287449393
cora 0 5
baseline 0.651255060728745
cora 0 6
baseline 0.6480161943319838
cora 0 7
baseline 0.6421052631578947
cora 0 8
baseline 0.6358704453441295
cora 0 9
baseline 0.6263967611336033
cora 0 10
baseline 0.617085020242915
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
cora 1 1
baseline 0.5942510121457489
cora 1 2
baseline 0.5970040485829959
cora 1 3
baseline 0.5952226720647774
cora 1 4
baseline 0.5949797570850202
cora 1 5
baseline 0.588744939271255
cora 1 6
baseline 0.5830769230769232
cora 1 7
baseline 0.5780566801619432
cora 1 8
baseline 0.5706072874493928
cora 1 9
baseline 0.5668016194331984
cora 1 10
baseline 0.5617004048582

baseline 0.6014998835677744
citeseer 5 10
baseline 0.603407643775606
-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
citeseer 6 1
baseline 0.5411072158871771
citeseer 6 2
baseline 0.5523608452979574
citeseer 6 3
baseline 0.5564612608154622
citeseer 6 4
baseline 0.5614195453321593
citeseer 6 5
baseline 0.5663789214009707
citeseer 6 6
baseline 0.5669507127835306
citeseer 6 7
baseline 0.5668546561974691
citeseer 6 8
baseline 0.5680944774740029
citeseer 6 9
baseline 0.5689528012865761
citeseer 6 10
baseline 0.56962083118055
-----------citeseer-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
citeseer 7 1
baseline 0.5128814974639606
citeseer 7 2
baseline 0.5174602856955733
citeseer 7 3
baseline 0.518223553511523
citeseer 7 4
baseline 0.5186055967515409
citeseer 7 5
baseline 0.5174616501357163
citeseer 7 6
baseline 0.5130727010093219
cit

KeyboardInterrupt: 

In [8]:
np.argmax(np.mean(accuracies,axis=1),axis=1),np.max(np.mean(accuracies,axis=1),axis=1)

(array([1, 1, 0, 0, 0]),
 array([0.59432389, 0.69617609, 0.65683706, 0.53346164, 0.20234025]))

In [6]:
np.mean(accuracies,axis=1)

array([[0.58711741, 0.59432389, 0.59308502, 0.58780567, 0.57948988,
        0.57013765, 0.56198381, 0.55289069, 0.54326316, 0.53516599],
       [0.68237691, 0.69617609, 0.6945861 , 0.68852014, 0.68135988,
        0.67429418, 0.66775474, 0.66042384, 0.65416459, 0.64815906],
       [0.65683706, 0.64679793, 0.62212293, 0.60109554, 0.58504607,
        0.57211157, 0.56068156, 0.54997602, 0.5399268 , 0.5307737 ]])

In [3]:
fin_accs = np.zeros((5,10))
i=0

# for data,num_iter in zip(['flickr'],[10]):
for data,num_iter in zip(['flickr'],[10]):


# for data,num_iter in zip(['citeseer','cora','pubmed'],[20,50,20]):

    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'flip',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

        lp = LP()
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             num_iter,
                             unlabeled_indices)

        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print(acc)
        fin_accs[i,seed] = acc
    i+=1

-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.6352806995311114
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7442656190596882
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.5529083766316056
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.547712583956406
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7225953618045875
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7175262957800025
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.5886452921049297
-----------flickr-----------
Loading labels...
Loading features

In [4]:
np.mean(fin_accs,axis=1)

array([0.64403751, 0.        , 0.        , 0.        , 0.        ])